In [6]:
import pandas as pd
from typing import Tuple, List, Dict
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os

In [7]:
import os
import csv

df_list = []

def read_files_and_write_to_csv(folder_path_1, folder_path_2, output_csv_path):    
    # Mở tệp CSV để ghi
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['NoiDung', 'label'])
        
        # Lặp qua các tệp trong thư mục
        for filename in os.listdir(folder_path_1):
            file_path_1 = os.path.join(folder_path_1, filename)
            
            # Kiểm tra xem tệp có phải là tệp văn bản không
            if os.path.isfile(file_path_1) and file_path_1.endswith('.txt'):
                with open(file_path_1, 'r', encoding='utf-8') as txt_file_1:
                    content = txt_file_1.read()
                    csv_writer.writerow([content, "notspam"])
                    
        for filename in os.listdir(folder_path_2):
            file_path_2 = os.path.join(folder_path_2, filename)
            
            # Kiểm tra xem tệp có phải là tệp văn bản không
            if os.path.isfile(file_path_2) and file_path_2.endswith('.txt'):
                with open(file_path_2, 'r', encoding='utf-8') as txt_file_2:
                    content = txt_file_2.read()
                    csv_writer.writerow([content, "spam"])            
    
    print(f"Đã ghi nội dung các tệp trong thư mục '{folder_path_1}'và '{folder_path_2} vào '{output_csv_path}'.")

# Đường dẫn đến thư mục chứa các tệp
folder_path_1 = 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/TrainData/notspam'
folder_path_2 = 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/TrainData/spam'
# Đường dẫn đến tệp CSV đích
output_csv_path = 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/pythonspamemail/x.csv'

read_files_and_write_to_csv(folder_path_1,folder_path_2, output_csv_path)


Đã ghi nội dung các tệp trong thư mục 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/TrainData/notspam'và 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/TrainData/spam vào 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/pythonspamemail/x.csv'.


In [8]:
df_list = pd.read_csv('C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/pythonspamemail/x.csv')

In [9]:
print(df_list)

df = df_list

                                               NoiDung    label
0    Subject: re : 2 . 882 s - > np np\n\n> date : ...  notspam
1    Subject: job announcement\n\njob announcement ...  notspam
2    Subject: translators needed women women !\n\na...  notspam
3    Subject: contributions solicited germanic gene...  notspam
4    Subject: celiac / oaxaca native literacy proje...  notspam
..                                                 ...      ...
206  Subject: lists software worldwide\n\norder for...     spam
207  Subject: zero down internet opportunity !\n\n$...     spam
208  Subject: re : free !\n\nhello , are offering f...     spam
209  Subject: \n\ncomes porn , site does n't mess a...     spam
210  Subject: even steal identity !\n\nare being in...     spam

[211 rows x 2 columns]


In [10]:
def text_preprocess(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()
    nostop=[word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]

    return nostop

In [11]:
spam_messages = df[df["label"] == "spam"]["NoiDung"]
notspam_messages = df[df["label"] == "notspam"]["NoiDung"]

In [12]:
import nltk
nltk.download('stopwords')

# Words in spam messages
spam_words = []
for each_message in spam_messages:
    spam_words += text_preprocess(each_message)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
df["NoiDung"] = df["NoiDung"].apply(text_preprocess)
df["NoiDung"] = df["NoiDung"].agg(lambda x: " ".join(map(str, x)))
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(df["NoiDung"])
messages_bow = bow_transformer.transform(df["NoiDung"])

In [14]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(211, 10343)


In [15]:
FactorResult = pd.factorize(df["label"])
df["label"] = FactorResult[0]
df.head(10)


,NoiDung,label
0,subject np np date sun dec est michael mmorse ...,0
1,subject job announcement job announcement depa...,0
2,subject translators needed women women posting...,0
3,subject contributions solicited germanic gener...,0
4,subject celiac oaxaca native literacy project ...,0
5,subject george aditjondro dr george aditjondro...,0
6,subject ua appeal arief budiman urgent action ...,0
7,subject designing dialects following appeared ...,0
8,subject language evolution context hello lingu...,0
9,subject quick note thank sent information conc...,0


In [16]:
# Split the dataset to train and test sets
msg_train, msg_test, label_train, label_test = train_test_split(
    messages_tfidf, df["label"], test_size=0.2
)

In [17]:
# Train an xgboost classifier
from xgboost import XGBClassifier

# Instantiate our model
clf = XGBClassifier()

# Fit the model to the training data
clf.fit(msg_train, label_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [18]:
# Make predictions
predict_train = clf.predict(msg_test)


In [20]:
no_labels = np.arange(0, 78)
df_noLabel = []

folder_path_3 = 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/TestData_nolabel/Testdata'

output_csv_path_2 = 'C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/pythonspamemail/y.csv'

with open(output_csv_path_2, 'w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['NoiDung', 'label'])
        
        # Lặp qua các tệp trong thư mục
        for filename in os.listdir(folder_path_3):
            file_path_3 = os.path.join(folder_path_3, filename)
            
            # Kiểm tra xem tệp có phải là tệp văn bản không
            if os.path.isfile(file_path_3) and file_path_3.endswith('.txt'):
                with open(file_path_3, 'r', encoding='utf-8') as txt_file:
                    content = txt_file.read()
                    csv_writer.writerow([content])


In [22]:
df_noLabel1 = pd.read_csv('C:/Users/Admin/OneDrive - ptit.edu.vn/Documents/BLT_PYTHON/BTL_FilteringSpamEmail/pythonspamemail/y.csv')
df_noLabel1.head(10)

,NoiDung,label
0,Subject: base generated adjuncts\n\ndoes anyon...,NaN
1,Subject: 4th nottingham international systemic...,NaN
2,Subject: salk insitute job\n\nresearch positio...,NaN
3,Subject: speaks languages ?\n\n> > : vicki fro...,NaN
4,Subject: syntax query\n\nmember tesl - l list ...,NaN
5,Subject: syntax textbooks\n\nneed order textbo...,NaN
6,Subject: re : 3 . 396 chomsky citations\n\nlin...,NaN
7,Subject: re : 3 . 396 chomsky citations\n\nlar...,NaN
8,Subject: chomsky citations\n\nalthough am fait...,NaN
9,Subject: re : 3 . 396 chomsky citations\n\nagr...,NaN


In [23]:
df_noLabel1["NoiDung"] = df_noLabel1["NoiDung"].apply(text_preprocess)
df_noLabel1["NoiDung"] = df_noLabel1["NoiDung"].agg(lambda x: " ".join(map(str, x)))
messages_bow_NL = bow_transformer.transform(df_noLabel1["NoiDung"])

In [24]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow_NL)

# Transform entire BoW into tf-idf corpus
messages_tfidf_NL = tfidf_transformer.transform(messages_bow_NL)


In [25]:
model_pred = clf.predict(messages_tfidf_NL)
model_pred = pd.DataFrame(model_pred)
model_pred = model_pred.replace({0: 'nospam', 1: 'spam'})
model_pred = model_pred.to_csv('ANSWER2.csv')
print(model_pred)



None
